In [11]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
#from tensorflow import keras
import matplotlib
from matplotlib import pyplot as plt
import imblearn
import gensim
import nltk
from pathlib import Path
#import torch
#import xgboost as xgb
import pickle
from nltk.stem import PorterStemmer
import tensorflow_hub as hub
import seaborn as sns
import os

In [2]:
#def adgroup_table():
rfc_model_pkl = open('rfc_model.pkl', 'rb')
rfc_model = pickle.load(rfc_model_pkl)
    #print("Loaded model :: ", rfc_model)

pickle_in = open("elmo_adgroups_array.pickle", "rb")
elmo_adgroups_array = pickle.load(pickle_in)

adgroup_keywords = pd.read_pickle('adgroup_keywords.pkl')

pred_adgroups = rfc_model.predict_proba(elmo_adgroups_array)[:,1]

adgroup_keywords['prob_clicks (%)'] = pred_adgroups * 100
adgroup_keywords.sort_values('prob_clicks (%)', ascending=False)


,ad_group,srch_keywords,prob_clicks (%)
0,3ds Max,mac autodesk version max mac os x autodesk for...,80.236827
94,Deep Learning Gpu,gpu deep deep cuda learning nvidia learning cl...,80.214464
249,Paul's Keywords,a computer desktop virtual machine service in ...,79.795246
360,Visual FX - Houdini,mac houdini engine mac program 15 3d in render...,79.793765
190,ML,hosting cloud of server cluster tensor startup...,79.783025
276,TensorFlow,gpu platform run on mac windows in good the se...,77.528239
265,Revit For Mac,revit autodesk a mac a version download on for...,77.231213
268,Run Windows On A Chromebook,convert virtual windows chromebook software on...,77.010176
31,Cloud Computer,a gpu computer virtual pc render rendering buy...,77.003495
187,Linux Desktop,desktop distros share pc top server the linux ...,76.863764


In [66]:
#adgroup_table()

In [3]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [4]:
text = ["Paperspace GPU parsec gradient"]

In [5]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
  #np.save('embeddings.pkl', embeddings)   
  #print(embeddings)
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [5]:
def click_prediction():

    text_array = np.asarray(text)

    list_text = [text_array[i:i+100] for i in range(0,text_array.shape[0],100)]
    elmo_text = [elmo_vectors(x) for x in list_text]
    elmo_text_array = np.concatenate(elmo_text, axis = 0)
    
    rfc_model_pkl = open('rfc_model.pkl', 'rb')
    rfc_model = pickle.load(rfc_model_pkl)
    #print("Loaded model :: ", rfc_model)
    pred_click = rfc_model.predict_proba(elmo_text_array)[:,1]
    return pred_click[0] * 100

In [6]:
click_prediction()

69.56219098840364

In [77]:
def click_pred():
    text_array = np.asarray(text)
    elmo_text = elmo_vectors(text_array)
    print(elmo_text.shape)
    #elmo_text_array = np.concatenate(elmo_text, axis = 0)
    rfc_model_pkl = open('rfc_model.pkl', 'rb')
    rfc_model = pickle.load(rfc_model_pkl)
    #print("Loaded model :: ", rfc_model)
    pred_click = rfc_model.predict_proba(elmo_text)[:,1]
    return pred_click[0] * 100

In [78]:
click_pred()

(1, 1024)


69.56219098840364

In [7]:
pickle_in = open("elmo_adgroups_array.pickle", "rb")
elmo_adgroups_array = pickle.load(pickle_in)
elmo_adgroups_array.shape


(386, 1024)

In [8]:
from sklearn.decomposition import PCA

#reduce down to 50 dim
pca = PCA(n_components=386) 
y = pca.fit_transform(elmo_adgroups_array)

from sklearn.manifold import TSNE

 # further reduce to 2 dim using t-SNE
y = TSNE(n_components=2).fit_transform(y)

In [9]:
adgroup_keywords = pd.read_pickle('adgroup_keywords.pkl')
df_y = pd.DataFrame(y, columns=['y0', 'y1'])
df = pd.concat([adgroup_keywords, df_y], axis=1)
df.to_pickle("adgroup_embeddings_vis.pkl")

df.head(5)


,ad_group,srch_keywords,y0,y1
0,3ds Max,mac autodesk version max mac os x autodesk for...,30.881138,5.249632
1,AI Cloud Amazon_broad,amazon artificial intelligence cloud aws ai,14.960670,-11.206009
2,AI Cloud Google_broad,artificial intelligence google cloud ai,14.372918,-10.460886
3,AI Cloud Gpu_broad,gpu ai cloud,-0.407174,-17.735321
4,AI Cloud Microsoft_broad,microsoft artificial intelligence cloud ai,14.656824,-10.674578


In [16]:
pickle_in = open("market_keywords.pickle", "rb")
market_keywords = pickle.load(pickle_in)
market_keywords.shape

(13949,)

In [17]:
# clean 
from nltk.stem import PorterStemmer
market_keywords.str.lower().str.replace('[!"#$%&()*+-/:;<=>?@[\\]^_`{|}~]',' ')
market_keywords.str.replace("[0-9]", ' ')
market_keywords.dropna()
market_keywords = market_keywords.apply(lambda x:' '.join(x.split()))
#unique_words = df_append.unique()

market_keywords = market_keywords.str.replace('+','')

ps = PorterStemmer()
market_keywords.map(ps.stem)
market_keywords.head()

# tokenize by letter
#df_date['stemmed'] = df_date['ad_matched_query'].apply(lambda x: [stemmer.stem(x) for y in x])

0                         2017 ai
1    2017 artificial intelligence
2     a 1 artificial intelligence
3                            a ai
4      a1 artificial intelligence
Name: Keyword, dtype: object

In [18]:
list_market_keywords = [market_keywords[i:i+1000] for i in range(0,market_keywords.shape[0],1000)]
elmo_market_keywords = [elmo_vectors(x) for x in list_market_keywords]
elmo_market_keywords_array = np.concatenate(elmo_market_keywords, axis = 0)

pickle_out = open("elmo_market_keywords_array.pickle","wb")
pickle.dump(elmo_market_keywords_array, pickle_out)
pickle_out.close()

In [19]:
pickle_in = open("elmo_market_keywords_array.pickle", "rb")
elmo_market_keywords_array = pickle.load(pickle_in)
elmo_market_keywords_array.shape

(13949, 1024)

In [57]:
#@title Sementic search
#@markdown Enter a set of words to find matching sentences. 'results_returned' can beused to modify the number of matching sentences retured. To view the code behind this cell, use the menu in the top right to unhide...
search_string = ["paperspace"] #@param {type:"string"}
results_returned = "3" #@param [1, 2, 3]

from sklearn.metrics.pairwise import cosine_similarity

search_array = np.asarray(search_string)
search_embedding = elmo_vectors(search_array)

cosine_similarities = pd.Series(cosine_similarity(search_embedding, elmo_market_keywords_array).flatten())
output =""
for  i,j in cosine_similarities.nlargest(int(results_returned)).iteritems():
  output +=''
  for i in market_keywords[i].split():
    if i in search_string:
      output += ""+str(i)+""
    else:
      output += " "+str(i)
  output += ', '
    
output = 'Results: '+output
print(output)

Results: paperspace tensorflow, paperspace desktop, paperspace server, 


In [79]:
g = tf.Graph()
with g.as_default():
  # We will be feeding 1D tensors of text into the graph.
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  embed = hub.Module('/storage/notebooks/tim/tfhub', trainable=True)
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()]) 
g.finalize()

# Create session and initialize.
session = tf.Session(graph=g)
session.run(init_op)

result = session.run(embedded_text, feed_dict={text_input: ["Hello world"]})